In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prensa digital

In [ ]:
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install spacy
!pip install pysentimiento==0.5.2rc4
!python -m spacy download es_core_news_sm
!pip install accelerate -U

In [ ]:
# Leer datos
import pandas as pd
import re
datos = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/2_outputs/1_prensa_digital/portales_digitales_marzo31_a_abril06.xlsx')
#datos['texto'][1]
#datos


# Nombres a buscar y sus respectivos patrones regex
nombres = {
    'Pablo Lemus Navarro': re.compile('Pablo Lemus|Lemus|PABLO LEMUS', re.IGNORECASE),
    'Claudia Delgadillo Gonzalez': re.compile('Claudia Delgadillo|CLAUDIA DELGADILLO', re.IGNORECASE),
    'Laura Lorena Haro Ramirez': re.compile('Laura Haro|LAURA HARO', re.IGNORECASE)
}

def extraer_contextos(row, n_palabras=15):
    nombre = row['persona_candidata']
    texto = row['texto']

    # Seleccionar el patrón de regex basado en el nombre
    patron = nombres.get(nombre)
    contextos = []  # Lista para almacenar todos los contextos encontrados

    if patron:
        for match in patron.finditer(texto):
            start, end = match.span()
            palabras_antes = ' '.join(texto[:start].split()[-n_palabras:])
            palabras_despues = ' '.join(texto[end:].split()[:n_palabras])
            contexto = f"{palabras_antes} {match.group()} {palabras_despues}".strip()
            contextos.append(contexto)

    return contextos  # Devolver la lista de contextos

# Aplicar la función a cada fila y crear una nueva columna 'contextos_encontrados' con los resultados
datos['contextos_encontrados'] = datos.apply(extraer_contextos, axis=1)
df_expandido = datos.explode('contextos_encontrados')
df_expandido = df_expandido.dropna(subset=['contextos_encontrados'])
df_expandido['programa_seccion'] = df_expandido['programa_seccion'].fillna('No aplica')
df_expandido['jerarquia'] = df_expandido['jerarquia'].fillna('No aplica')


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
import pandas as pd
import spacy
from pysentimiento import create_analyzer

# Asumiendo que df_digital_s2 y las bibliotecas ya están cargadas

# Cargar Spacy para el idioma español
nlp = spacy.load("es_core_news_sm")

# Crear el analizador de sentimiento para español
analyzer = create_analyzer(task="sentiment", lang="es")

# Contar el total de textos para informar del progreso
total_textos = len(df_expandido)
print(f"Procesando {total_textos} textos...")

# Listas para almacenar los resultados de análisis
noticia_id = []
sentimientos = []
prob_negs = []
prob_neus = []
prob_poss = []

# Iterar sobre cada texto/noticia en el DataFrame
for index, row in df_expandido.iterrows():
    text = row['contextos_encontrados']
    print(f"Procesando texto {index+1} de {total_textos}...")  # Informar sobre el progreso

    # Analizar el sentimiento del texto completo
    analisis = analyzer.predict(text)

    # Almacenar los resultados
    noticia_id.append(index)
    sentimientos.append(analisis.output)
    prob_negs.append(analisis.probas['NEG'])
    prob_neus.append(analisis.probas['NEU'])
    prob_poss.append(analisis.probas['POS'])

# Añadir las columnas de análisis al DataFrame original
df_expandido['Sentimiento'] = sentimientos
df_expandido['Prob_NEG'] = prob_negs
df_expandido['Prob_NEU'] = prob_neus
df_expandido['Prob_POS'] = prob_poss

# Mostrar o guardar el DataFrame actualizado
df_expandido
# Guardar los datos
df_expandido.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/digital_sent_oraciones.csv',index=False)

In [ ]:
import pandas as pd
# Leer el df de sentimiento y agrupar por nota
sent_5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/digital_sent_oraciones.csv')

sent_5 = sent_5.drop(['contextos_encontrados','Sentimiento','horario'],axis=1)

columnas_para_agrupar = sent_5.columns[:-3]

In [ ]:
import pandas as pd
# Leer el df de sentimiento y agrupar por nota
sent_5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/digital_sent_oraciones.csv')

sent_5 = sent_5.drop(['contextos_encontrados','Sentimiento','horario'],axis=1)

columnas_para_agrupar = sent_5.columns[:-3]
columnas_para_agrupar

# Nombres de las columnas para las cuales quieres calcular el promedio
columnas_promedio = sent_5.columns[-3:]
columnas_promedio
# Nombres de las columnas por las cuales agrupar (todas excepto las últimas 3)
columnas_para_agrupar = list(sent_5.columns[:-3])
columnas_para_agrupar
# Usar groupby y agg para calcular el promedio de las últimas 3 columnas
#mientras se agrupa por el resto de columnas
df_agrupado = sent_5.groupby(list(columnas_para_agrupar)).agg(
    Negativo = ('Prob_NEG','mean'),
    Neutral = ('Prob_NEU','mean'),
    Positivo = ('Prob_POS','mean')

  )
df_agrupado

df_agrupado['sentimiento'] = df_agrupado[['Negativo', 'Positivo', 'Neutral']].idxmax(axis=1)
df_agrupado = df_agrupado.drop(['Negativo','Neutral','Positivo'],axis=1).reset_index()
df_agrupado.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/digital_sentimiento.csv' ,index=False)

In [ ]:
# Leer las oraciones y obtener las estadisticas
import pandas as pd
oraciones_digital = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/digital_sent_oraciones.csv')


oraciones_digital['mujer'] = oraciones_digital['persona_candidata'].apply(lambda x: 1 if x in ['Laura Lorena Haro Ramirez', 'Claudia Delgadillo Gonzalez'] else 0)
oraciones_digital['hombre'] = oraciones_digital['persona_candidata'].apply(lambda x: 1 if x in ['Pablo Lemus Navarro'] else 0)
oraciones_digital['sexo'] = oraciones_digital['persona_candidata'].apply(lambda x: 'Hombre' if x in ['Pablo Lemus Navarro'] else 'Mujer')
oraciones_digital

conteo_sentimiento = pd.crosstab(oraciones_digital['persona_candidata'], oraciones_digital['Sentimiento'])

# Ahora, si aún deseas agregar las medias de 'Prob_NEG', 'Prob_NEU', y 'Prob_POS', puedes hacerlo por separado
medias = oraciones_digital.groupby('persona_candidata').agg({
    'Prob_NEG': 'mean',
    'Prob_NEU': 'mean',
    'Prob_POS': 'mean'
})

# Finalmente, puedes unir ambos DataFrames para tener el conteo y las medias juntas
resultado_final = conteo_sentimiento.join(medias).reset_index()
resultado_final['mean_neg'] = 100*resultado_final['NEG'] / (resultado_final['NEG']+resultado_final['NEU']+resultado_final['POS'])
resultado_final['mean_neu'] = 100*resultado_final['NEU'] / (resultado_final['NEG']+resultado_final['NEU']+resultado_final['POS'])
resultado_final['mean_pos'] = 100*resultado_final['POS'] / (resultado_final['NEG']+resultado_final['NEU']+resultado_final['POS'])

resultado_final

In [ ]:
conteo_sentimiento = pd.crosstab(oraciones_digital['sexo'], oraciones_digital['Sentimiento'])

# Ahora, si aún deseas agregar las medias de 'Prob_NEG', 'Prob_NEU', y 'Prob_POS', puedes hacerlo por separado
medias = oraciones_digital.groupby('sexo').agg({
    'Prob_NEG': 'mean',
    'Prob_NEU': 'mean',
    'Prob_POS': 'mean'
})

# Finalmente, puedes unir ambos DataFrames para tener el conteo y las medias juntas
resultado_final_sex = conteo_sentimiento.join(medias).reset_index()
resultado_final_sex['mean_neg'] = 100*resultado_final_sex['NEG'] / (resultado_final_sex['NEG']+resultado_final_sex['NEU']+resultado_final_sex['POS'])
resultado_final_sex['mean_neu'] = 100*resultado_final_sex['NEU'] / (resultado_final_sex['NEG']+resultado_final_sex['NEU']+resultado_final_sex['POS'])
resultado_final_sex['mean_pos'] = 100*resultado_final_sex['POS'] / (resultado_final_sex['NEG']+resultado_final_sex['NEU']+resultado_final_sex['POS'])



resultado_final_sex.rename(columns={'sexo': 'persona_candidata'}, inplace=True)
resultado_final_sex

In [ ]:
general = pd.DataFrame(oraciones_digital.value_counts('Sentimiento')).reset_index()
general.columns = [ 'Sentimiento','Cuenta']

# Calcular el total de cuentas
total_cuentas = sum(general['Cuenta'])

# Calcular el porcentaje (promedio) de cada sentimiento respecto al total
general['Porcentaje'] = (general['Cuenta'] / total_cuentas) * 100

print(general)
general.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/analisis_sentimiento_digital_general.csv',index=False)

total_cuentas

In [ ]:
sentimientos_final_fisica = pd.concat([resultado_final_sex,resultado_final], ignore_index=True)
sentimientos_final_fisica.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/analisis_sentimiento_digital.csv',index=False)
sentimientos_final_fisica

# **Semana 6 Gubernaturas**

In [ ]:
# Leer datos
import pandas as pd
import re
datos = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/2_outputs/1_prensa_digital/Reporte 2/gubernatura_reporte_2.xlsx')
#datos['texto'][1]
#datos


# Nombres a buscar y sus respectivos patrones regex
nombres = {
    'Pablo Lemus Navarro': re.compile('Pablo Lemus|Lemus|PABLO LEMUS', re.IGNORECASE),
    'Claudia Delgadillo Gonzalez': re.compile('Claudia Delgadillo|CLAUDIA DELGADILLO', re.IGNORECASE),
    'Laura Lorena Haro Ramirez': re.compile('Laura Haro|LAURA HARO', re.IGNORECASE)
}

def extraer_contextos(row, n_palabras=15):
    nombre = row['persona_candidata']
    texto = row['texto']

    # Seleccionar el patrón de regex basado en el nombre
    patron = nombres.get(nombre)
    contextos = []  # Lista para almacenar todos los contextos encontrados

    if patron:
        for match in patron.finditer(texto):
            start, end = match.span()
            palabras_antes = ' '.join(texto[:start].split()[-n_palabras:])
            palabras_despues = ' '.join(texto[end:].split()[:n_palabras])
            contexto = f"{palabras_antes} {match.group()} {palabras_despues}".strip()
            contextos.append(contexto)

    return contextos  # Devolver la lista de contextos

# Aplicar la función a cada fila y crear una nueva columna 'contextos_encontrados' con los resultados
datos['contextos_encontrados'] = datos.apply(extraer_contextos, axis=1)
df_expandido = datos.explode('contextos_encontrados')
df_expandido = df_expandido.dropna(subset=['contextos_encontrados'])
df_expandido['programa_seccion'] = df_expandido['programa_seccion'].fillna('No aplica')
df_expandido['jerarquia'] = df_expandido['jerarquia'].fillna('No aplica')


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_expandido

,fecha,candidatura,persona_candidata,genero,edad_18a35,pueblo_originario,diversidad_sexual,discapacidad,residente_extranjero,medio,...,prd,pt,pvem,mc,morena,hagamos,futuro,fcj,shh,contextos_encontrados
0,2024-03-11,Gubernatura,Laura Lorena Haro Ramirez,mujer,0,0,0,0,0,El Informador,...,1,0,0,0,0,0,0,1,0,Pide Laura Haro analizar a policía vial por co...
0,2024-03-11,Gubernatura,Laura Lorena Haro Ramirez,mujer,0,0,0,0,0,El Informador,...,1,0,0,0,0,0,0,1,0,del Estado por la coalición PRI - PAN - PRD “F...
0,2024-03-11,Gubernatura,Laura Lorena Haro Ramirez,mujer,0,0,0,0,0,El Informador,...,1,0,0,0,0,0,0,1,0,de que la seguridad vial haya pasado a manos d...
1,2024-03-11,Gubernatura,Claudia Delgadillo Gonzalez,mujer,0,0,0,0,0,El Informador,...,0,1,1,0,1,1,1,0,1,"“Se les acabó el negocio del Recrea”, asegura ..."
1,2024-03-11,Gubernatura,Claudia Delgadillo Gonzalez,mujer,0,0,0,0,0,El Informador,...,0,1,1,0,1,1,1,0,1,"de educación básica no sólo seguirán, sino que..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2024-03-12,Gubernatura,Claudia Delgadillo Gonzalez,mujer,0,0,0,0,0,Semanario 7 Días,...,0,0,0,0,0,0,0,0,0,Claudia Delgadillo llegó con las propuestas pa...
269,2024-03-12,Gubernatura,Claudia Delgadillo Gonzalez,mujer,0,0,0,0,0,Semanario 7 Días,...,0,0,0,0,0,0,0,0,0,Claudia Delgadillo llegó con las propuestas pa...
270,2024-03-12,Gubernatura,Claudia Delgadillo Gonzalez,mujer,0,0,0,0,0,Semanario 7 Días,...,0,0,0,0,0,0,0,0,0,Claudia Delgadillo llegó con las propuestas pa...
271,2024-03-12,Gubernatura,Claudia Delgadillo Gonzalez,mujer,0,0,0,0,0,Semanario 7 Días,...,0,0,0,0,0,0,0,0,0,Claudia Delgadillo llegó con las propuestas pa...


In [ ]:
import pandas as pd
import spacy
from pysentimiento import create_analyzer

# Cargar Spacy para el idioma español
nlp = spacy.load("es_core_news_sm")

# Crear el analizador de sentimiento para español
analyzer = create_analyzer(task="sentiment", lang="es")

# Contar el total de textos para informar del progreso
total_textos = len(df_expandido)
print(f"Procesando {total_textos} textos...")

# Listas para almacenar los resultados de análisis
noticia_id = []
sentimientos = []
prob_negs = []
prob_neus = []
prob_poss = []

# Iterar sobre cada texto/noticia en el DataFrame
for index, row in df_expandido.iterrows():
    text = row['contextos_encontrados']
    print(f"Procesando texto {index+1} de {total_textos}...")  # Informar sobre el progreso

    # Analizar el sentimiento del texto completo
    analisis = analyzer.predict(text)

    # Almacenar los resultados
    noticia_id.append(index)
    sentimientos.append(analisis.output)
    prob_negs.append(analisis.probas['NEG'])
    prob_neus.append(analisis.probas['NEU'])
    prob_poss.append(analisis.probas['POS'])

# Añadir las columnas de análisis al DataFrame original
df_expandido['Sentimiento'] = sentimientos
df_expandido['Prob_NEG'] = prob_negs
df_expandido['Prob_NEU'] = prob_neus
df_expandido['Prob_POS'] = prob_poss

# Mostrar o guardar el DataFrame actualizado
df_expandido
# Guardar los datos
df_expandido.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_2/gubernatura/digital_sent_oraciones.csv',index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Procesando 568 textos...
Procesando texto 1 de 568...
Procesando texto 1 de 568...
Procesando texto 1 de 568...
Procesando texto 2 de 568...
Procesando texto 2 de 568...
Procesando texto 2 de 568...
Procesando texto 2 de 568...
Procesando texto 4 de 568...
Procesando texto 4 de 568...
Procesando texto 4 de 568...
Procesando texto 5 de 568...
Procesando texto 5 de 568...
Procesando texto 5 de 568...
Procesando texto 5 de 568...
Procesando texto 5 de 568...
Procesando texto 6 de 568...
Procesando texto 7 de 568...
Procesando texto 7 de 568...
Procesando texto 7 de 568...
Procesando texto 8 de 568...
Procesando texto 9 de 568...
Procesando texto 9 de 568...
Procesando texto 10 de 568...
Procesando texto 11 de 568...
Procesando texto 11 de 568...
Procesando texto 12 de 568...
Procesando texto 12 de 568...
Procesando texto 12 de 568...
Procesando texto 13 de 568...
Procesando texto 13 de 568...
Procesando texto 13 de 568...
Procesando texto 13 de 568...
Procesando texto 15 de 568...
Procesa

In [ ]:
import pandas as pd
# Leer el df de sentimiento y agrupar por nota
sent_6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_2/gubernatura/digital_sent_oraciones.csv')

sent_6 = sent_6.drop(['contextos_encontrados','Sentimiento','horario'],axis=1)

columnas_para_agrupar = sent_6.columns[:-3]


# Nombres de las columnas por las cuales agrupar (todas excepto las últimas 3)
columnas_para_agrupar = list(sent_6.columns[:-3])
columnas_para_agrupar
# Usar groupby y agg para calcular el promedio de las últimas 3 columnas
#mientras se agrupa por el resto de columnas
df_agrupado = sent_6.groupby(list(columnas_para_agrupar)).agg(
    Negativo = ('Prob_NEG','mean'),
    Neutral = ('Prob_NEU','mean'),
    Positivo = ('Prob_POS','mean')

  )
df_agrupado

df_agrupado['sentimiento'] = df_agrupado[['Negativo', 'Positivo', 'Neutral']].idxmax(axis=1)
df_agrupado = df_agrupado.drop(['Negativo','Neutral','Positivo'],axis=1).reset_index()
df_agrupado.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_2/gubernatura/digital_sentimiento.csv' ,index=False)

In [ ]:
# Leer las oraciones y obtener las estadisticas
import pandas as pd
oraciones_digital = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_2/gubernatura/digital_sent_oraciones.csv')


oraciones_digital['mujer'] = oraciones_digital['persona_candidata'].apply(lambda x: 1 if x in ['Laura Lorena Haro Ramirez', 'Claudia Delgadillo Gonzalez'] else 0)
oraciones_digital['hombre'] = oraciones_digital['persona_candidata'].apply(lambda x: 1 if x in ['Pablo Lemus Navarro'] else 0)
oraciones_digital['sexo'] = oraciones_digital['persona_candidata'].apply(lambda x: 'Hombre' if x in ['Pablo Lemus Navarro'] else 'Mujer')
oraciones_digital.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fecha                  568 non-null    object 
 1   candidatura            568 non-null    object 
 2   persona_candidata      568 non-null    object 
 3   genero                 568 non-null    object 
 4   edad_18a35             568 non-null    int64  
 5   pueblo_originario      568 non-null    int64  
 6   diversidad_sexual      568 non-null    int64  
 7   discapacidad           568 non-null    int64  
 8   residente_extranjero   568 non-null    int64  
 9   medio                  535 non-null    object 
 10  tipo_medio             568 non-null    object 
 11  programa_seccion       568 non-null    object 
 12  horario                0 non-null      float64
 13  autoria                550 non-null    object 
 14  jerarquia              568 non-null    object 
 15  genero

In [ ]:
conteo_sentimiento = pd.crosstab(oraciones_digital['persona_candidata'], oraciones_digital['Sentimiento'])

# Ahora, si aún deseas agregar las medias de 'Prob_NEG', 'Prob_NEU', y 'Prob_POS', puedes hacerlo por separado
medias = oraciones_digital.groupby('persona_candidata').agg({
    'Prob_NEG': 'mean',
    'Prob_NEU': 'mean',
    'Prob_POS': 'mean'
})

# Finalmente, puedes unir ambos DataFrames para tener el conteo y las medias juntas
resultado_final = conteo_sentimiento.join(medias).reset_index()
resultado_final['mean_neg'] = 100*resultado_final['NEG'] / (resultado_final['NEG']+resultado_final['NEU']+resultado_final['POS'])
resultado_final['mean_neu'] = 100*resultado_final['NEU'] / (resultado_final['NEG']+resultado_final['NEU']+resultado_final['POS'])
resultado_final['mean_pos'] = 100*resultado_final['POS'] / (resultado_final['NEG']+resultado_final['NEU']+resultado_final['POS'])

resultado_final

,persona_candidata,NEG,NEU,POS,Prob_NEG,Prob_NEU,Prob_POS,mean_neg,mean_neu,mean_pos
0,Claudia Delgadillo Gonzalez,52,73,54,0.321791,0.387046,0.291163,29.050279,40.782123,30.167598
1,Laura Lorena Haro Ramirez,40,97,30,0.284679,0.444700,0.270622,23.952096,58.083832,17.964072
2,Pablo Lemus Navarro,42,127,53,0.268060,0.446848,0.285092,18.918919,57.207207,23.873874


In [ ]:
conteo_sentimiento = pd.crosstab(oraciones_digital['sexo'], oraciones_digital['Sentimiento'])

# Ahora, si aún deseas agregar las medias de 'Prob_NEG', 'Prob_NEU', y 'Prob_POS', puedes hacerlo por separado
medias = oraciones_digital.groupby('sexo').agg({
    'Prob_NEG': 'mean',
    'Prob_NEU': 'mean',
    'Prob_POS': 'mean'
})

# Finalmente, puedes unir ambos DataFrames para tener el conteo y las medias juntas
resultado_final_sex = conteo_sentimiento.join(medias).reset_index()
resultado_final_sex['mean_neg'] = 100*resultado_final_sex['NEG'] / (resultado_final_sex['NEG']+resultado_final_sex['NEU']+resultado_final_sex['POS'])
resultado_final_sex['mean_neu'] = 100*resultado_final_sex['NEU'] / (resultado_final_sex['NEG']+resultado_final_sex['NEU']+resultado_final_sex['POS'])
resultado_final_sex['mean_pos'] = 100*resultado_final_sex['POS'] / (resultado_final_sex['NEG']+resultado_final_sex['NEU']+resultado_final_sex['POS'])



resultado_final_sex.rename(columns={'sexo': 'persona_candidata'}, inplace=True)
resultado_final_sex

,persona_candidata,NEG,NEU,POS,Prob_NEG,Prob_NEU,Prob_POS,mean_neg,mean_neu,mean_pos
0,Hombre,42,127,53,0.268060,0.446848,0.285092,18.918919,57.207207,23.873874
1,Mujer,92,170,84,0.303878,0.414873,0.281249,26.589595,49.132948,24.277457


In [ ]:
general = pd.DataFrame(oraciones_digital.value_counts('Sentimiento')).reset_index()
general.columns = [ 'Sentimiento','Cuenta']

# Calcular el total de cuentas
total_cuentas = sum(general['Cuenta'])

# Calcular el porcentaje (promedio) de cada sentimiento respecto al total
general['Porcentaje'] = (general['Cuenta'] / total_cuentas) * 100

print(general)
general.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_2/gubernatura/analisis_sentimiento_digital_general.csv',index=False)

total_cuentas

  Sentimiento  Cuenta  Porcentaje
0         NEU     297   52.288732
1         POS     137   24.119718
2         NEG     134   23.591549


568

In [ ]:
sentimientos_final = pd.concat([resultado_final_sex,resultado_final], ignore_index=True)
sentimientos_final.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_2/gubernatura/analisis_sentimiento_digital.csv',index=False)
sentimientos_final

,persona_candidata,NEG,NEU,POS,Prob_NEG,Prob_NEU,Prob_POS,mean_neg,mean_neu,mean_pos
0,Hombre,42,127,53,0.268060,0.446848,0.285092,18.918919,57.207207,23.873874
1,Mujer,92,170,84,0.303878,0.414873,0.281249,26.589595,49.132948,24.277457
2,Claudia Delgadillo Gonzalez,52,73,54,0.321791,0.387046,0.291163,29.050279,40.782123,30.167598
3,Laura Lorena Haro Ramirez,40,97,30,0.284679,0.444700,0.270622,23.952096,58.083832,17.964072
4,Pablo Lemus Navarro,42,127,53,0.268060,0.446848,0.285092,18.918919,57.207207,23.873874
